In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline ()
%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams

rcParams['figure.figsize'] = (9, 6)

UsageError: unrecognized arguments: ()


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [4]:
train = pd.read_csv('C:/Netology/Machine_Learning/Lecture_9/data/train.csv', parse_dates=['timestamp'])

print("Shape of dataset", train.shape,"\n")
print("Basic information of our data", train.info(),"\n")

test = pd.read_csv('C:/Netology/Machine_Learning/Lecture_9/data/test.csv', parse_dates=['timestamp'])
macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract", 
              "micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate", 
              "income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]

macro = pd.read_csv('C:/Netology/Machine_Learning/Lecture_9/data/macro.csv', parse_dates=['timestamp'], usecols=['timestamp'] + macro_cols)

Shape of dataset (30471, 292) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Columns: 292 entries, id to price_doc
dtypes: datetime64[ns](1), float64(119), int64(157), object(15)
memory usage: 67.9+ MB
Basic information of our data None 



In [5]:
test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [6]:
train.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [7]:
data = pd.concat([train, test])

data = pd.merge_ordered(data, macro, on='timestamp', how='left')

print(data.shape)

(38133, 305)


In [8]:
data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,average_provision_of_build_contract,micex_rgbi_tr,micex_cbi_tr,deposits_rate,mortgage_value,mortgage_rate,income_per_cap,rent_price_4+room_bus,museum_visitis_per_100_cap,apartment_build
0,1,2011-08-20,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,5.76,131.16,204.78,4.1,323275,11.84,42688.6,136.11,1240.0,23587.0
1,2,2011-08-23,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,5.76,131.45,204.92,4.1,323275,11.84,42688.6,136.11,1240.0,23587.0
2,3,2011-08-27,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,5.76,131.08,204.84,4.1,323275,11.84,42688.6,136.11,1240.0,23587.0
3,4,2011-09-01,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,5.76,131.45,205.42,4.0,386388,11.92,40311.3,155.22,1240.0,23587.0
4,5,2011-09-05,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,5.76,131.19,198.24,4.0,386388,11.92,40311.3,155.22,1240.0,23587.0


In [9]:
print(pd.set_option("display.max_rows", 999))

print("Basic description of our data ", data.describe())

None
Basic description of our data                   id       full_sq       life_sq         floor     max_floor  \
count  38133.000000  38133.000000  30574.000000  37966.000000  28561.000000   
mean   19068.933994     54.111172     34.033460      7.667123     12.567592   
std    11008.303803     35.171162     47.581529      5.276156      6.730496   
min        1.000000      0.000000      0.000000      0.000000      0.000000   
25%     9536.000000     38.900000     20.000000      3.000000      9.000000   
50%    19069.000000     50.000000     30.000000      7.000000     12.000000   
75%    28602.000000     63.000000     43.000000     11.000000     17.000000   
max    38135.000000   5326.000000   7478.000000     77.000000    117.000000   

           material    build_year      num_room      kitch_sq         state  \
count  28561.000000  2.347900e+04  28561.000000  28561.000000  23880.000000   
mean       1.834390  2.716785e+03      1.900844      6.543995      2.071650   
std        1.49

In [10]:
# удаляем не релевантные столбцы

data.drop(['id'], axis=1, inplace=True)

In [11]:
# Давайте возьмем столбец Year из нашей транзакции timestamp 


data["trans_year"]=data.timestamp.dt.year

month_year = (data.timestamp.dt.month + data.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
data['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (data.timestamp.dt.weekofyear + data.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
data['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
data['month'] = data.timestamp.dt.month
data['dow'] = data.timestamp.dt.dayofweek

# Remove timestamp column (may overfit the model in train)
data.drop(['timestamp'], axis=1, inplace=True)

In [12]:
data = data[data.build_year != 20052009.0]

data.build_year = np.where(data.build_year<1691, np.nan, data.build_year)

In [13]:
data.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,mortgage_rate,income_per_cap,rent_price_4+room_bus,museum_visitis_per_100_cap,apartment_build,trans_year,month_year_cnt,week_year_cnt,month,dow
0,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,1,8,5
1,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,2,8,1
2,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,2,8,5
3,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.92,40311.3,155.22,1240.0,23587.0,2011,39,1,9,3
4,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.92,40311.3,155.22,1240.0,23587.0,2011,39,5,9,0


In [14]:
# проверим типы данных

for col, type_col in zip(data.columns, data.dtypes):
    print(f'Столбец {col} тип данных {type_col}')

Столбец full_sq тип данных float64
Столбец life_sq тип данных float64
Столбец floor тип данных float64
Столбец max_floor тип данных float64
Столбец material тип данных float64
Столбец build_year тип данных float64
Столбец num_room тип данных float64
Столбец kitch_sq тип данных float64
Столбец state тип данных float64
Столбец product_type тип данных object
Столбец sub_area тип данных object
Столбец area_m тип данных float64
Столбец raion_popul тип данных int64
Столбец green_zone_part тип данных float64
Столбец indust_part тип данных float64
Столбец children_preschool тип данных int64
Столбец preschool_quota тип данных float64
Столбец preschool_education_centers_raion тип данных int64
Столбец children_school тип данных int64
Столбец school_quota тип данных float64
Столбец school_education_centers_raion тип данных int64
Столбец school_education_centers_top_20_raion тип данных int64
Столбец hospital_beds_raion тип данных float64
Столбец healthcare_centers_raion тип данных int64
Столбец uni

In [15]:
# Проверим на наличие дубликатов

duplicate_rows_df = data[data.duplicated()]
print('Число дублирующихся строк: ', duplicate_rows_df.shape)

Число дублирующихся строк:  (10, 308)


In [16]:
# удалим дубликаты строк

data = data.drop_duplicates()
data.head(5)

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,mortgage_rate,income_per_cap,rent_price_4+room_bus,museum_visitis_per_100_cap,apartment_build,trans_year,month_year_cnt,week_year_cnt,month,dow
0,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,1,8,5
1,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,2,8,1
2,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,2,8,5
3,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.92,40311.3,155.22,1240.0,23587.0,2011,39,1,9,3
4,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.92,40311.3,155.22,1240.0,23587.0,2011,39,5,9,0


In [17]:
# Проверим количество строк в столбцах

for col, i in zip(data.columns, data.count()):
    print(f'Столбец {col} количество строк {i}')

Столбец full_sq количество строк 38122
Столбец life_sq количество строк 30570
Столбец floor количество строк 37955
Столбец max_floor количество строк 28555
Столбец material количество строк 28555
Столбец build_year количество строк 22015
Столбец num_room количество строк 28555
Столбец kitch_sq количество строк 28555
Столбец state количество строк 23875
Столбец product_type количество строк 38089
Столбец sub_area количество строк 38122
Столбец area_m количество строк 38122
Столбец raion_popul количество строк 38122
Столбец green_zone_part количество строк 38122
Столбец indust_part количество строк 38122
Столбец children_preschool количество строк 38122
Столбец preschool_quota количество строк 29843
Столбец preschool_education_centers_raion количество строк 38122
Столбец children_school количество строк 38122
Столбец school_quota количество строк 29847
Столбец school_education_centers_raion количество строк 38122
Столбец school_education_centers_top_20_raion количество строк 38122
Столбе

In [18]:
# процент пропусков в столбцах, где есть пропуски

i = 0

print('===========================================================')
print('Пропуски в данных:')
for column, item in zip(data.columns, (data.isna().mean() * 100).round(2)):
    if item != 0:
        print(f'Название столбца: {column}, наличие пропусков {item} %')
        i += 1
print('===========================================================')
print(f'Количество стобцов с пропусками {i}')

Пропуски в данных:
Название столбца: life_sq, наличие пропусков 19.81 %
Название столбца: floor, наличие пропусков 0.44 %
Название столбца: max_floor, наличие пропусков 25.1 %
Название столбца: material, наличие пропусков 25.1 %
Название столбца: build_year, наличие пропусков 42.25 %
Название столбца: num_room, наличие пропусков 25.1 %
Название столбца: kitch_sq, наличие пропусков 25.1 %
Название столбца: state, наличие пропусков 37.37 %
Название столбца: product_type, наличие пропусков 0.09 %
Название столбца: preschool_quota, наличие пропусков 21.72 %
Название столбца: school_quota, наличие пропусков 21.71 %
Название столбца: hospital_beds_raion, наличие пропусков 46.83 %
Название столбца: raion_build_count_with_material_info, наличие пропусков 16.27 %
Название столбца: build_count_block, наличие пропусков 16.27 %
Название столбца: build_count_wood, наличие пропусков 16.27 %
Название столбца: build_count_frame, наличие пропусков 16.27 %
Название столбца: build_count_brick, наличие пр

In [19]:
# количество пропусков в столбцах, где есть пропуски

print('===========================================================')
print('Количество пропусков в данных:')
print(data.isnull().sum()[data.isnull().sum().values != 0])
print('===========================================================')

Количество пропусков в данных:
life_sq                                   7552
floor                                      167
max_floor                                 9567
material                                  9567
build_year                               16107
num_room                                  9567
kitch_sq                                  9567
state                                    14247
product_type                                33
preschool_quota                           8279
school_quota                              8275
hospital_beds_raion                      17852
raion_build_count_with_material_info      6204
build_count_block                         6204
build_count_wood                          6204
build_count_frame                         6204
build_count_brick                         6204
build_count_monolith                      6204
build_count_panel                         6204
build_count_foam                          6204
build_count_slag             

In [20]:
# предположим, мы хотим оставить только те строки, в которых как минимум 280/308 значений заполнено


update_data = data.dropna(thresh=280)

In [21]:
print(f'Количество теряемых данных при исключении строк, в которых как минимум 280/308 значений не заполнено: {100 - len(update_data)/len(data) * 100} %')

Количество теряемых данных при исключении строк, в которых как минимум 280/308 значений не заполнено: 8.530507318608684 %


In [22]:
new_data = update_data.copy()

In [23]:
new_data.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,mortgage_rate,income_per_cap,rent_price_4+room_bus,museum_visitis_per_100_cap,apartment_build,trans_year,month_year_cnt,week_year_cnt,month,dow
0,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,1,8,5
1,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,2,8,1
2,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.84,42688.6,136.11,1240.0,23587.0,2011,3,2,8,5
3,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.92,40311.3,155.22,1240.0,23587.0,2011,39,1,9,3
4,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,...,11.92,40311.3,155.22,1240.0,23587.0,2011,39,5,9,0


In [24]:
# проверим как много нулевых данных в нашем датасете

null_columns=new_data.columns[new_data.isnull().any()]

new_data[null_columns].isnull().sum()

life_sq                                   5735
floor                                      131
max_floor                                 8115
material                                  8115
build_year                               13518
num_room                                  8115
kitch_sq                                  8115
state                                    12206
product_type                                26
preschool_quota                           5027
school_quota                              5023
hospital_beds_raion                      14603
raion_build_count_with_material_info      2952
build_count_block                         2952
build_count_wood                          2952
build_count_frame                         2952
build_count_brick                         2952
build_count_monolith                      2952
build_count_panel                         2952
build_count_foam                          2952
build_count_slag                          2952
build_count_m

In [25]:
#Находим категориальные признаки

cat_feat = list(new_data.dtypes[new_data.dtypes == object].index)

print(cat_feat)

['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']


In [26]:
#Находим числовые признаки

int_feat = list(new_data.dtypes[new_data.dtypes != object].index)

print(int_feat)

['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_quota', 'preschool_education_centers_raion', 'children_school', 'school_quota', 'school_education_centers_raion', 'school_education_centers_top_20_raion', 'hospital_beds_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', 'culture_objects_top_25_raion', 'shopping_centers_raion', 'office_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'bu

In [27]:
# Разбиваем данные на категориальные и числовые

data_cat = new_data[cat_feat]

data_int = new_data.drop(cat_feat, axis=1)

### 1. Заполним пропуски нулевыми значениями и обучим модель линейной регрессии

In [28]:
list_score_test = []
list_score_train = []

In [29]:
# заполняем пропуски

new_data[data_cat.columns] = new_data[data_cat.columns].fillna('NaN')

new_data[data_int.columns] = new_data[data_int.columns].fillna(0)

In [30]:
# обрабатываем категориальные переменные

from sklearn.preprocessing import LabelEncoder 

le = LabelEncoder()

for i  in data_cat.columns:
    if len(new_data[i].unique()) < 8:
        new_data = pd.get_dummies(new_data, columns=[i])
    else:
        le.fit(new_data[i])
        new_data[i] = le.transform(new_data[i])

In [31]:
# разделим наши данные

X_train, x_test, Y_train, y_test = train_test_split(new_data.loc[:, new_data.columns != 'price_doc'], new_data['price_doc'], test_size=0.3, random_state = 42)

In [32]:
lin_reg = LinearRegression()

model = lin_reg.fit(X_train, Y_train)

print('Weights: {}'.format(model.coef_))
print('Bias: {}'.format(model.intercept_))

pred_values = model.predict(x_test)

list_score_test.append(model.score(x_test, y_test))
list_score_train.append(model.score(X_train, Y_train))


print('Точность модели на тестовой выборке: {}'.format(model.score(x_test, y_test)))
print('Точность модели на тренировочной выборке: {}'.format(model.score(X_train, Y_train)))
print('Среднеквадратическая ошибка: {}'.format(mean_squared_error(pred_values, y_test)))
print('Абсолютная ошибка: {}'.format(mean_absolute_error(pred_values, y_test)))

Weights: [ 8.83830534e+04  9.91235149e+03  4.51857995e+04  1.53285289e+04
 -1.80945496e+04 -1.05132362e+02  1.38369823e+05 -2.21470240e+02
  2.85002276e+05 -1.35794726e+03 -2.68572338e-02 -2.80504211e+06
 -8.92478741e+05 -4.50592391e+05  1.32810216e+06  1.83462885e+02
  5.06264140e+04  1.75966483e+06 -2.46582585e+02  2.27212101e+05
 -1.93973556e+05  1.81353388e+02 -2.96175718e+04 -1.04364261e+05
 -4.11666630e+04  1.20442272e+05  1.33806869e+04 -8.33153356e+04
  1.09181688e+04 -1.69466979e+06  1.69470922e+06  1.69466744e+06
 -5.13927642e+06  7.94350796e+06  7.94071681e+06  2.98894863e+06
 -1.83821786e+05 -1.84037148e+05 -6.54714132e+05  3.45973706e+06
  3.45986623e+06  1.32810218e+06 -2.65703536e+06 -2.65525639e+06
  1.75966481e+06 -3.51807731e+06 -3.51906429e+06 -3.34462297e+06
  3.34391158e+06  3.34728461e+06  4.39853066e+06 -4.39876759e+06
 -4.39846443e+06  1.52933006e+06 -1.52831339e+06 -1.52866320e+06
  1.49242488e+05  9.35555984e+03  3.12080462e+04 -1.30504578e+04
  1.84038238e+04

### 2. Заменим пропуски и попробуем реализовать полиномиальную регрессиюи попробуем реализовать полиномиальную регрессию:   
* в Buildyear как среднее значение build_year в этой подобласти
* в max_floor как среднее значение max_floor в этой подобласти

In [33]:
new_data_2 = update_data.copy()

In [34]:
new_data_2.build_year = new_data_2.groupby(["sub_area"])['build_year'].apply(lambda x: x.fillna(x.mean()))
new_data_2["build_year"]=new_data_2["build_year"].astype("int64")

print(f'Давайте посмотрим, сколько нулевых значений осталось в build_year: {new_data_2.build_year.isnull().sum()}.')

Давайте посмотрим, сколько нулевых значений осталось в build_year: 0.


In [35]:
#Даже после замены на среднее значение build_year равно нулю, что совершенно не имеет значения.
#следовательно, без группировки заменим эти значения на средние.

new_data_2.build_year=np.where(new_data_2.build_year<1691, np.nan, new_data_2.build_year)
new_data_2.build_year.fillna(new_data_2.build_year.mean(), inplace=True)

In [36]:
new_data_2.build_year.isnull().sum()

0

In [37]:
new_data_2.max_floor = new_data_2.groupby(["sub_area"])['max_floor'].apply(lambda x: x.fillna(x.mean()))
new_data_2["max_floor"]= new_data_2["max_floor"].astype("int64")

print(f'Давайте посмотрим, сколько нулевых значений осталось в max_floor: {new_data_2.max_floor.isnull().sum()}.')

Давайте посмотрим, сколько нулевых значений осталось в max_floor: 0.


In [38]:
# заполняем пропуски

new_data_2[data_cat.columns] = new_data_2[data_cat.columns].fillna('NaN')

new_data_2[data_int.columns] = new_data_2[data_int.columns].fillna(0)

In [40]:
# обрабатываем категориальные переменные

from sklearn.preprocessing import LabelEncoder 

le = LabelEncoder()

for i  in data_cat.columns:
    if len(new_data_2[i].unique()) < 8:
        new_data_2 = pd.get_dummies(new_data_2, columns=[i])
    else:
        le.fit(new_data_2[i])
        new_data_2[i] = le.transform(new_data_2[i])

In [43]:
poly = PolynomialFeatures(degree = 2) 
X_poly = poly.fit_transform(X_train)
model = lin_reg.fit(X_poly, Y_train)

print('Weights: {}'.format(model.coef_))
print('Bias: {}'.format(model.intercept_))

pred_values = model.predict(x_test)

list_score_test.append(model.score(x_test, y_test))
list_score_train.append(model.score(X_poly, Y_train))


print('Точность модели на тестовой выборке: {}'.format(model.score(x_test, y_test)))
print('Точность модели на тренировочной выборке: {}'.format(model.score(X_train, Y_train)))
print('Среднеквадратическая ошибка: {}'.format(mean_squared_error(pred_values, y_test)))
print('Абсолютная ошибка: {}'.format(mean_absolute_error(pred_values, y_test)))

MemoryError: Unable to allocate 9.69 GiB for an array with shape (24409, 53301) and data type float64